## 구글 이미지 크롤링

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.request
import os
import time
import os
from PIL import Image

def convert_images_to_pdf(directory_path, pdf_path):
    # 디렉토리에서 이미지 파일을 찾음
    files = os.listdir(directory_path)
    image_files = [f for f in files if f.endswith('.jpg') or f.endswith('.png')]

    # 이미지를 PDF로 변환
    with Image.open(os.path.join(directory_path, image_files[0])) as img:
        img.save(pdf_path, save_all=True, append_images=[Image.open(os.path.join(directory_path, f)) for f in image_files[1:]])


def collect_image(search_word, extract_img_count):
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--ignore-ssl-errors')
    options.add_argument('disable-infobars')
    options.add_argument('--disable-extensions')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')

    driver = webdriver.Chrome('chromedriver', options=options)
    driver.implicitly_wait(3)

    # 구글 이미지 검색 페이지 로드
    url = 'https://www.google.com/search?q=' + search_word + '&source=lnms&tbm=isch'
    driver.get(url)

    # 페이지 스크롤 다운
    for i in range(10):
        driver.execute_script("window.scrollBy(0, 10000);")
        time.sleep(1)

    # 이미지 추출
    imgs = driver.find_elements(By.CSS_SELECTOR, 'img.rg_i')

    # 추출된 이미지에서 src 속성값 추출
    img_srcs = []
    for img in imgs:
        src = img.get_attribute('src')
        if src and 'http' in src:
            img_srcs.append(src)

    # 이미지 다운로드
    if len(img_srcs) > extract_img_count:
        img_srcs = img_srcs[:extract_img_count]

    for i, src in enumerate(img_srcs):
        print('Downloading image', i+1)
        try:
            # 이미지 url 열기
            with urllib.request.urlopen(src) as url:
                # 이미지 파일 확장자 추출
                file_ext = os.path.splitext(src)[1]
                # 이미지 파일 저장
                with open(f"{search_word}_{i}.jpg", "wb") as f:  # modify file extension to jpg
                    f.write(url.read())
        except Exception as e:
            print(e)

    driver.close()

if __name__ == '__main__':
    collect_image('서울 노을 명소', 200)